# Data Science Academy

## Fundamentos de Linguagem Python Para Análise de Dados e Data Science

### Capítulo 16 - Series Temporais

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())